Добрый день!  
Задание по итогам второго вебинара.  
Приложен ноутбук, в котором реализованы функции для генирации из большого датасета в меньшую его копию. Вам нужно перенести функции из этого ноутбука в класс датасет и сделать следующее: 
Сгенерировать меньший датасет из 8-10 классов движения  
Обучить уже существующую модель (предварительно проанализировав какие параметры модели нужно изменить)  
Изменить модель: посмотреть зависимость от количества LSTM модулей в нашей модели    
Сгенерировать другой датасет с меньшим количеством “кадров” в серии и сравнить улучшилось или ухудшилось качество предсказания.  
Провести несколько таких итераций, дать свою оценку уменьшению и увеличению кадров, назвать оптимальное, на ваш взгляд, их количество. Желательно сделать так, чтобы длина последовательности передавалась как атрибут класса.


In [119]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torch import optim
import time
import os
import math
from pprint import pprint
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [27]:
data_path = r"D:\train\skiliton\nturgbd_skeletons_s001_to_s017/"
#### список отсутсвующих элементов так же будет доступен 
broken_files_path = r"D:\train\skiliton\NTU_RGBD_samples_with_missing_skeletons.txt"

A1. drink water.   
 
A3. brushing teeth.  

A4. brushing hair.  

A8. sitting down.

A22. cheer up. 

A23. hand waving.  

A24. kicking something.  

A25. reach into pocket.  

A26. hopping (one foot jumping).  

A28. make a phone call/answer phone.  

A30. typing on a keyboard.  
 





In [2]:
# training_subjects = list(range(0, 50)) #количество людей выполняющих действия
 #классы которые будем использовать для обучения, полный список прдставлен тут https://github.com/shahroudy/NTURGB-D
# training_cameras = [1, 2, 3] 

# num_joint = 25
# max_frame = 1000 # Длина отрезка которую мы вычленяем из большого датасета

In [60]:
class Skeleton_Dataset(Dataset):
    training_subjects = list(range(0, 50)) #количество людей выполняющих действия
    training_cameras = [1, 2, 3] 
    num_joint = 25 #Количество точек в скелете
    max_frame = 5000 # Длина отрезка которую мы вычленяем из большого датасета
    training_classes = [1 , 3, 4, 8, 22,23,24,25,28, 30]
    max_body = 1
    
    def __init__(self, data_path,broken_files_path,chonk_len):
        self.data_path = data_path
        self.broken_files_path = broken_files_path
        self.chonk_len = chonk_len
        working_files_with_labels, action_classes = self.read_data(data_path, broken_files_path)
        self.dict_labels = {v: k for k, v in action_classes.items()}
        data = []
        labels = []
        ##########################################################################
        numbers = {x: 0 for x in range(len(action_classes))} #####
        ##################################################################
        for file in working_files_with_labels:
            frames_blocks, label = self.create_coords_blocks(file)
            if label != [] and numbers[label[0]] <= 150:
                numbers[label[0]] = numbers[label[0]] + len(label)
                data = data + frames_blocks
                labels = labels + label
        data_np = np.asarray(data)
        
        self.labels_np = np.asarray(labels)
        self.data_sq = data_np.reshape(len(data_np), -1)
        
    
    
    def __len__(self):
        return len(self.data_sq)  
    
    
    
    def __getitem__(self, idx):
        item = self.data_sq[idx].reshape(self.chonk_len,self.num_joint*len(self.training_cameras))
        label = self.labels_np[idx]
        return (item, label)
        
            

    def create_coords_blocks(self,test_file):   
        chonk_len = self.chonk_len
        frame_counter = 0
        new_labels = []
        new_frames = []
        blocks = []

        test_frames = self.read_xyz(data_path + test_file[0])[0]
        label = test_file[1]
        slice_len = chonk_len * int(len(test_frames)/chonk_len)


        for index in range(len(test_frames[:slice_len])):
            frame_counter += 1
            new_frames.append(test_frames[index].flatten())
            if frame_counter == chonk_len:
                frame_counter = 0
                blocks.append(np.array(new_frames))
                new_labels = new_labels + [label]
                new_frames = []
                
        return blocks, new_labels

            
    
    def read_data(self,data_path, broken_files_path):
        labels = []
        files = []
        action_classes = {}
        counter = 0
        files_counter = {}

        with open(broken_files_path, 'r') as f:
            broken_files = f.read().split("\n")

        raw_files = os.listdir(data_path)
        num_frames = 0

        for filename in raw_files:
            if filename not in broken_files:
                action_class = int(filename[filename.find('A') + 1:filename.find('A') + 4])
                subject_id = int(filename[filename.find('P') + 1:filename.find('P') + 4])
                camera_id = int(filename[filename.find('C') + 1:filename.find('C') + 4])
                if action_class in self.training_classes and camera_id in self.training_cameras:  #and subject_id in training_subjects:
                    if action_class in action_classes:
                        if files_counter[action_class] < 120:
                            files.append([filename,action_classes[action_class]])
                            files_counter[action_class] = files_counter[action_class] + 1
                    else:
                        action_classes.update({action_class : counter})
                        files_counter.update({action_class : 1})
                        counter+=1
                        files.append([filename,action_classes[action_class]])

        return files, action_classes        
            
        
    # Функция фильтр для того, что бы найти только координаты x,y,z(т.к. в датасете
    # хранится на порядок больше информации, нежели только координаты x,y,z)
    # (Остальные данные нам и не нужны, т.к. у нас нет ик-камер)
    def read_skeleton_filter(self,file):
        with open(file, 'r') as f:
            skeleton_sequence = {}
            skeleton_sequence['numFrame'] = int(f.readline())
            skeleton_sequence['frameInfo'] = []
            for t in range(skeleton_sequence['numFrame']):
                frame_info = {}
                frame_info['numBody'] = int(f.readline())
                frame_info['bodyInfo'] = []

                for m in range(frame_info['numBody']):
                    body_info = {}
                    body_info_key = [
                        'bodyID', 'clipedEdges', 'handLeftConfidence',
                        'handLeftState', 'handRightConfidence', 'handRightState',
                        'isResticted', 'leanX', 'leanY', 'trackingState'
                    ]
                    body_info = {
                        k: float(v)
                        for k, v in zip(body_info_key, f.readline().split())
                    }
                    body_info['numJoint'] = int(f.readline())
                    body_info['jointInfo'] = []
                    for v in range(body_info['numJoint']):
                        joint_info_key = [
                            'x', 'y', 'z', 'depthX', 'depthY', 'colorX', 'colorY',
                            'orientationW', 'orientationX', 'orientationY',
                            'orientationZ', 'trackingState'
                        ]
                        joint_info = {
                            k: float(v)
                            for k, v in zip(joint_info_key, f.readline().split())
                        }
                        body_info['jointInfo'].append(joint_info)
                    frame_info['bodyInfo'].append(body_info)
                skeleton_sequence['frameInfo'].append(frame_info)

        return skeleton_sequence

    # Здесь мы используем нашу функцию фильр и оформляем дату в x,y,z-cкоординаты
    def read_xyz(self,file):
        seq_info = self.read_skeleton_filter(file)
        data = np.zeros((self.max_body, seq_info['numFrame'], self.num_joint, 3))
        for n, f in enumerate(seq_info['frameInfo']):
            for m, b in enumerate(f['bodyInfo']):
                for j, v in enumerate(b['jointInfo']):
                    if m < self.max_body and j < self.num_joint:
                        data[m, n, j, :] = [v['x'], v['y'], v['z']]

                    else:
                        pass

        return data
         

In [61]:
data_set = Skeleton_Dataset(data_path = data_path,broken_files_path = broken_files_path,chonk_len = 45)

In [62]:
a,b = data_set[13]

In [67]:
a.shape,b

((45, 75), 8)

In [65]:
np.unique(data_set.labels_np)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [75]:
data_set.dict_labels

{0: 1, 1: 3, 2: 4, 3: 8, 4: 22, 5: 23, 6: 24, 7: 25, 8: 28, 9: 30}

In [71]:
len_test = int(len(data_set)*0.2)
len_train = len(data_set) - len_test
train_dataset, test_dataset = torch.utils.data.random_split(data_set, [len_train,len_test])
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = 1, shuffle=True)

In [72]:
a = iter(train_loader).next()
a

[tensor([[[-1.0219e-01, -1.1035e-01,  3.4137e+00,  ..., -6.5885e-02,
           -2.1314e-01,  3.1837e+00],
          [-1.0112e-01, -1.1153e-01,  3.4111e+00,  ..., -6.0752e-02,
           -1.9878e-01,  3.1862e+00],
          [-1.0183e-01, -1.1183e-01,  3.4108e+00,  ..., -6.1654e-02,
           -1.9991e-01,  3.1803e+00],
          ...,
          [-9.6978e-02, -1.1760e-01,  3.4276e+00,  ..., -3.4680e-01,
            4.8367e-01,  3.0554e+00],
          [-9.7029e-02, -1.1783e-01,  3.4281e+00,  ..., -3.2723e-01,
            5.2649e-01,  3.0556e+00],
          [-9.7087e-02, -1.1863e-01,  3.4287e+00,  ..., -3.0558e-01,
            5.5446e-01,  3.0515e+00]],
 
         [[-2.6959e-01,  7.6978e-02,  3.0914e+00,  ..., -1.5451e-01,
            7.9324e-01,  2.9282e+00],
          [-2.6925e-01,  7.6976e-02,  3.0914e+00,  ..., -1.6216e-01,
            7.9348e-01,  2.9339e+00],
          [-2.6932e-01,  7.6844e-02,  3.0915e+00,  ..., -1.5431e-01,
            7.9777e-01,  2.9461e+00],
          ...,
    

In [73]:
class LSTM_net(nn.Module):
    # Для того, что бы инициализировать LSTM нам нужно указать:
    # input_dim - размерность входного тензора. тензор входит в формате (seq_len, batch, input_dim)
    # (batch_size, seq, inp_dim) - if batch_first=True
    # hidden_dim - размерность вектора состояния h
    # output_dim - размерность выхода
    # layer_num - количество скрытых слоев в сети
    def __init__(self,input_dim,hidden_dim,output_dim,layer_num):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim,layer_num,batch_first=True)
        self.dr = torch.nn.Dropout2d(0.1)
        self.fc = torch.nn.Linear(hidden_dim,output_dim)
        
        
    def forward(self,inputs):
        x = inputs
        lstm_out,(hn,cn) = self.lstm(x)
        out = self.fc(lstm_out[:,-1,:])
        return out

In [79]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
#     print(output.topk(5))
    return data_set.dict_labels[category_i], category_i

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [110]:
param = {
'n_epoch' : 600,
'n_hidden' : 128,
'n_layer' : 2,
'learning_rate' : 0.0007,
'n_joints' : data_set.num_joint*len(data_set.training_cameras),
'n_categories' : len(data_set.dict_labels),
'print':False,   
}

In [111]:
def testing(**param):
    rnn = LSTM_net(param['n_joints'],param['n_hidden'],param['n_categories'],param ['n_layer'])
    rnn.to(device)
    criterion = nn.CrossEntropyLoss()
    learning_rate = param['learning_rate']
    optimizer = optim.SGD(rnn.parameters(),lr=learning_rate,momentum=0.9)

    all_losses = []
    start = time.time()
    counter = 0
    for epoch in range(param['n_epoch']):  
        current_loss = 0
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):

            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()

            output = rnn(inputs.float())
            loss = criterion(output, labels.long())
            loss.backward()
            optimizer.step() 


            current_loss += loss.item()
            category = data_set.dict_labels[int(labels[0])]

            if param['print']:
                if counter % 500 == 0:
                    guess, guess_i = categoryFromOutput(output)
                    correct = '✓' if guess == category else '✗ (%s)' % category
                    print('epoch : %d iter : %d (%s) %.4f  / %s %s' % (epoch, i, timeSince(start), loss, guess, correct))


            counter = counter + 1
        if counter % 100 == 0:
            all_losses.append(current_loss / 25)
            current_loss = 0

        total = 0
        right = 0
        counter = 0

    rnn.eval()
    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            counter = counter + 1
            inputs, labels = data[0].to(device), data[1].to(device)  
            output = rnn(inputs.float())
            guess, guess_i = categoryFromOutput(output)
            category = data_set.dict_labels[int(labels[0])]

            if guess == category:
                right = right + 1

    acc = (100 * right / counter)
    if param['print']:
        print('Accuracy of the network:  ',  acc)
    return acc    
    
    




In [112]:
testing(**param)

73.10344827586206

Начнем тестирование

In [121]:
d = {
'n_epoch' : [],
'n_hidden' : [],
'n_layer' : [],
'n_categories' : [],
'chonk_len' :[],
'accuracy':[]
}
i = 0
for chonk_len in [15,25,35,45,50,55,60]:
    data_set = Skeleton_Dataset(data_path = data_path,broken_files_path = broken_files_path,chonk_len = chonk_len)
    len_test = int(len(data_set)*0.2)
    len_train = len(data_set) - len_test
    train_dataset, test_dataset = torch.utils.data.random_split(data_set, [len_train,len_test])
    train_loader = DataLoader(train_dataset, batch_size = 64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size = 1, shuffle=True)
    for n_layer in [2,4,6,8]:
        for n_hidden in [64,128,256]:
            param = {
                'n_epoch' : 600,
                'n_hidden' : n_hidden,
                'n_layer' : n_layer,
                'learning_rate' : 0.0007,
                'n_joints' : data_set.num_joint*len(data_set.training_cameras),
                'n_categories' : len(data_set.dict_labels),
                'print':False,   
                }
            acc =  testing(**param)
            d['n_epoch'].append(param['n_epoch'])
            d['n_hidden'].append(n_hidden)
            d['n_layer'].append(n_layer)
            d['n_categories'].append(param['n_categories'])
            d['chonk_len'].append(chonk_len)
            d['accuracy'].append(acc)
            print(i)
            pprint(d)
            i+=1
            
result = pd.DataFrame(d)  


0
{'accuracy': [57.377049180327866],
 'chonk_len': [15],
 'n_categories': [10],
 'n_epoch': [600],
 'n_hidden': [64],
 'n_layer': [2]}
1
{'accuracy': [57.377049180327866, 63.9344262295082],
 'chonk_len': [15, 15],
 'n_categories': [10, 10],
 'n_epoch': [600, 600],
 'n_hidden': [64, 128],
 'n_layer': [2, 2]}
2
{'accuracy': [57.377049180327866, 63.9344262295082, 64.91803278688525],
 'chonk_len': [15, 15, 15],
 'n_categories': [10, 10, 10],
 'n_epoch': [600, 600, 600],
 'n_hidden': [64, 128, 256],
 'n_layer': [2, 2, 2]}
3
{'accuracy': [57.377049180327866,
              63.9344262295082,
              64.91803278688525,
              26.885245901639344],
 'chonk_len': [15, 15, 15, 15],
 'n_categories': [10, 10, 10, 10],
 'n_epoch': [600, 600, 600, 600],
 'n_hidden': [64, 128, 256, 64],
 'n_layer': [2, 2, 2, 4]}
4
{'accuracy': [57.377049180327866,
              63.9344262295082,
              64.91803278688525,
              26.885245901639344,
              23.278688524590162],
 'chonk_len

16
{'accuracy': [57.377049180327866,
              63.9344262295082,
              64.91803278688525,
              26.885245901639344,
              23.278688524590162,
              23.278688524590162,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              57.755775577557756,
              61.71617161716171,
              59.40594059405941,
              16.831683168316832,
              20.792079207920793],
 'chonk_len': [15,
               15,
               15,
               15,
               15,
               15,
               15,
               15,
               15,
               15,
               15,
               15,
               25,
               25,
               25,
               25,
               25],
 'n_categories': [10,
                  10,
                  10,
                  10,
                  10,
 

20
{'accuracy': [57.377049180327866,
              63.9344262295082,
              64.91803278688525,
              26.885245901639344,
              23.278688524590162,
              23.278688524590162,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              57.755775577557756,
              61.71617161716171,
              59.40594059405941,
              16.831683168316832,
              20.792079207920793,
              14.851485148514852,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858],
 'chonk_len': [15,
               15,
               15,
               15,
               15,
               15,
               15,
               15,
               15,
               15,
               15,
               15,
               25,
               25,
               25,
               25,
 

24
{'accuracy': [57.377049180327866,
              63.9344262295082,
              64.91803278688525,
              26.885245901639344,
              23.278688524590162,
              23.278688524590162,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              57.755775577557756,
              61.71617161716171,
              59.40594059405941,
              16.831683168316832,
              20.792079207920793,
              14.851485148514852,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              68.64686468646865],
 'chonk_len': [15,
               15,
               15,
               15,
               15,
               15,
               15,
               15,
               15,
     

27
{'accuracy': [57.377049180327866,
              63.9344262295082,
              64.91803278688525,
              26.885245901639344,
              23.278688524590162,
              23.278688524590162,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              57.755775577557756,
              61.71617161716171,
              59.40594059405941,
              16.831683168316832,
              20.792079207920793,
              14.851485148514852,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              68.64686468646865,
              68.31683168316832,
              66.66666666666667,
              33.00330033003301],
 'chonk_len': [15,
               15,
               15,
               15,
 

...

80
{'accuracy': [57.377049180327866,
              63.9344262295082,
              64.91803278688525,
              26.885245901639344,
              23.278688524590162,
              23.278688524590162,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              57.755775577557756,
              61.71617161716171,
              59.40594059405941,
              16.831683168316832,
              20.792079207920793,
              14.851485148514852,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              68.64686468646865,
              68.31683168316832,
              66.66666666666667,
              33.00330033003301,
              26.73267326732673,
              29.042904290429043,
           

81
{'accuracy': [57.377049180327866,
              63.9344262295082,
              64.91803278688525,
              26.885245901639344,
              23.278688524590162,
              23.278688524590162,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              57.755775577557756,
              61.71617161716171,
              59.40594059405941,
              16.831683168316832,
              20.792079207920793,
              14.851485148514852,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              68.64686468646865,
              68.31683168316832,
              66.66666666666667,
              33.00330033003301,
              26.73267326732673,
              29.042904290429043,
           

82
{'accuracy': [57.377049180327866,
              63.9344262295082,
              64.91803278688525,
              26.885245901639344,
              23.278688524590162,
              23.278688524590162,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              57.755775577557756,
              61.71617161716171,
              59.40594059405941,
              16.831683168316832,
              20.792079207920793,
              14.851485148514852,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              68.64686468646865,
              68.31683168316832,
              66.66666666666667,
              33.00330033003301,
              26.73267326732673,
              29.042904290429043,
           

83
{'accuracy': [57.377049180327866,
              63.9344262295082,
              64.91803278688525,
              26.885245901639344,
              23.278688524590162,
              23.278688524590162,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              8.19672131147541,
              57.755775577557756,
              61.71617161716171,
              59.40594059405941,
              16.831683168316832,
              20.792079207920793,
              14.851485148514852,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              8.58085808580858,
              68.64686468646865,
              68.31683168316832,
              66.66666666666667,
              33.00330033003301,
              26.73267326732673,
              29.042904290429043,
           

,n_epoch,n_hidden,n_layer,n_categories,chonk_len,accuracy
61,600,128,2,10,55,73.134328
49,600,128,2,10,50,71.071429
62,600,256,2,10,55,70.522388
74,600,256,2,10,60,68.800000
24,600,64,2,10,35,68.646865
...,...,...,...,...,...,...
34,600,128,8,10,35,6.270627
31,600,128,6,10,35,6.270627
33,600,64,8,10,35,6.270627
30,600,64,6,10,35,6.270627


In [122]:
result.sort_values('accuracy',ascending=False)

,n_epoch,n_hidden,n_layer,n_categories,chonk_len,accuracy
61,600,128,2,10,55,73.134328
49,600,128,2,10,50,71.071429
62,600,256,2,10,55,70.522388
74,600,256,2,10,60,68.800000
24,600,64,2,10,35,68.646865
...,...,...,...,...,...,...
34,600,128,8,10,35,6.270627
31,600,128,6,10,35,6.270627
33,600,64,8,10,35,6.270627
30,600,64,6,10,35,6.270627


Как видно лучшие результаты получились при использовании только 2 словевбn_hidden 128 и 256 при  chonk_len  50 и 55  
С рочсто количества слоев качестов падает

In [123]:
result.sort_values('accuracy',ascending=False).head(20)

,n_epoch,n_hidden,n_layer,n_categories,chonk_len,accuracy
61,600,128,2,10,55,73.134328
49,600,128,2,10,50,71.071429
62,600,256,2,10,55,70.522388
74,600,256,2,10,60,68.800000
24,600,64,2,10,35,68.646865
25,600,128,2,10,35,68.316832
26,600,256,2,10,35,66.666667
48,600,64,2,10,50,65.000000
2,600,256,2,10,15,64.918033
38,600,256,2,10,45,64.827586
